<a href="https://colab.research.google.com/github/apester/IME/blob/main/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Multi-label Classification using ResNet (Python Notebook)
!pip install tensorflow  matplotlib
# Cell 1: Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Cell 2: Dataset Preparation (using CIFAR-10 as an example)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Artificially create a multi-label dataset
num_classes = 10
y_train_multilabel = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test_multilabel = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

# Introduce random additional labels to simulate multi-label classification
def add_random_labels(labels, num_classes, probability=0.2):
    new_labels = labels.copy()
    for label in new_labels:
        extra_labels = np.random.rand(num_classes) < probability
        label[extra_labels] = 1
    return new_labels

# Apply to dataset
y_train_multilabel = add_random_labels(y_train_multilabel, num_classes)
y_test_multilabel = add_random_labels(y_test_multilabel, num_classes)

# Normalize images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
# Cell 3: Build the ResNet Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet initially
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Cell 4: Train the Model
history = model.fit(
    x_train, y_train_multilabel,
    epochs=10,
    batch_size=64,
    validation_split=0.2
)

In [ ]:
# Cell 5: Evaluate the Model
loss, accuracy = model.evaluate(x_test, y_test_multilabel)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

# Cell 6: Visualize Training History
plt.figure(figsize=(12,4))

In [ ]:
# Plot loss
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
# Plot accuracy
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
